In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 10.1 MB/s 
     |████████████████████████████████| 209 kB 72.7 MB/s 
     |████████████████████████████████| 78 kB 8.7 MB/s 
     |████████████████████████████████| 49 kB 8.0 MB/s 
     |████████████████████████████████| 112 kB 70.4 MB/s 
     |████████████████████████████████| 146 kB 69.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=ba7c135a4a9cb0dac28019e29d768403b97ccd9a93354c0c707283a2acb7196b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [92]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
from torchsummary import summary
import numpy as np

In [125]:
DEVICE = torch.device("cuda")
BATCHSIZE = 128
CLASSES = 100
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 300
N_VALID_EXAMPLES = BATCHSIZE * 100

In [132]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers_Conv = trial.suggest_int("n_layers_conv", 1, 4)
    n_layers_Dense = trial.suggest_int("n_layers_dense", 1, 3)
    layers = []

    #print(f"{n_layers_Conv} layers Conv and {n_layers_Dense} Dense")

    in_channel = 3
    width_height = 32

    # convolution layers
    for i in range(n_layers_Conv):
      n_filters = trial.suggest_categorical("filters_{}".format(i), [16, 32, 64, 128])
      n_kernel_size = trial.suggest_categorical("kernel_size_{}".format(i), [3, 5])
      n_stride = trial.suggest_categorical("strides_{}".format(i), [1, 2])

      # for kernel > image
      if (n_kernel_size > width_height):
        continue

      layers.append(nn.Conv2d(in_channels = in_channel, out_channels= n_filters, kernel_size= n_kernel_size, stride= n_stride,padding='valid'))
      layers.append(nn.ReLU())

      # get size of convs
      TMP = ((width_height - n_kernel_size) // n_stride) + 1
      out_shape = (TMP, TMP, n_filters)

      #print(f"in_channels = {in_channel}\nout_channels = {n_filters}\nkernel_size = {n_kernel_size}\nstride = {n_stride}\nOutput Shape = {out_shape}")
      #print('----------------------------------------------------------------')

      width_height = TMP
      in_channel = n_filters

    # Flatten layer
    layers.append(nn.Flatten())
    

    in_features = out_shape[0] * out_shape[1] * out_shape[2]
    #in_features = out_shape[0]
    for i in range(n_layers_Dense):
        out_features = trial.suggest_int("n_units_{}".format(i), 64, 128)
        layers.append(nn.Linear(in_features, out_features))

        #print(f"in_features = {in_features}\nout_features = {out_features}")
        #print('----------------------------------------------------------------')

        layers.append(nn.ReLU())

        in_features = out_features

    # last Dense layer    
    layers.append(nn.Linear(in_features, CLASSES))
    #print(f"in_features = {in_features}\nout_features = {CLASSES}")
    #print('----------------------------------------------------------------')

    #layers.append(nn.LogSoftmax(dim=1))
    layers.append(nn.Softmax(dim = 1))

    network = nn.Sequential(*layers)
    #print('----------------------------------------------------------------')
    #summary(network, (3, 32, 32))
    #print('----------------------------------------------------------------')

    return network

In [122]:
def get_CIFAR100():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader

In [121]:
def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the CIFAR100 dataset.
    train_loader, valid_loader = get_CIFAR100()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):

            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break
            
            data, target = data.to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        #if trial.should_prune():
            #raise optuna.exceptions.TrialPruned()

    return accuracy

# Q4_B
## Run 1

In [130]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, timeout=600)

    #pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    #print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-06-16 14:37:48,413] A new study created in memory with name: no-name-2a821240-963c-44da-9a96-487e2adeec2a


Files already downloaded and verified


[I 2022-06-16 14:38:54,858] Trial 0 finished with value: 0.01 and parameters: {'n_layers_conv': 4, 'n_layers_dense': 3, 'filters_0': 16, 'kernel_size_0': 3, 'strides_0': 1, 'filters_1': 16, 'kernel_size_1': 5, 'strides_1': 2, 'filters_2': 128, 'kernel_size_2': 3, 'strides_2': 2, 'filters_3': 32, 'kernel_size_3': 5, 'strides_3': 2, 'n_units_0': 128, 'dropout_0': 0.495118504751762, 'n_units_1': 109, 'dropout_1': 0.49642993965099325, 'n_units_2': 73, 'dropout_2': 0.28303117688504814, 'optimizer': 'RMSprop', 'lr': 0.0026043548118035347}. Best is trial 0 with value: 0.01.


Files already downloaded and verified


[I 2022-06-16 14:40:18,511] Trial 1 finished with value: 0.0657 and parameters: {'n_layers_conv': 3, 'n_layers_dense': 1, 'filters_0': 32, 'kernel_size_0': 3, 'strides_0': 1, 'filters_1': 32, 'kernel_size_1': 3, 'strides_1': 1, 'filters_2': 128, 'kernel_size_2': 5, 'strides_2': 1, 'n_units_0': 82, 'dropout_0': 0.2585625969071337, 'optimizer': 'Adam', 'lr': 0.00010247187950542526}. Best is trial 1 with value: 0.0657.


...

Files already downloaded and verified


[I 2022-06-16 14:47:11,630] Trial 7 finished with value: 0.01 and parameters: {'n_layers_conv': 4, 'n_layers_dense': 3, 'filters_0': 32, 'kernel_size_0': 5, 'strides_0': 1, 'filters_1': 32, 'kernel_size_1': 3, 'strides_1': 1, 'filters_2': 32, 'kernel_size_2': 3, 'strides_2': 2, 'filters_3': 64, 'kernel_size_3': 5, 'strides_3': 2, 'n_units_0': 106, 'dropout_0': 0.28346881620527525, 'n_units_1': 103, 'dropout_1': 0.3783633031867276, 'n_units_2': 68, 'dropout_2': 0.35402994584556824, 'optimizer': 'SGD', 'lr': 0.0074126136260227755}. Best is trial 1 with value: 0.0657.


Files already downloaded and verified


[I 2022-06-16 14:48:15,478] Trial 8 finished with value: 0.01 and parameters: {'n_layers_conv': 3, 'n_layers_dense': 3, 'filters_0': 32, 'kernel_size_0': 3, 'strides_0': 2, 'filters_1': 16, 'kernel_size_1': 3, 'strides_1': 1, 'filters_2': 128, 'kernel_size_2': 3, 'strides_2': 2, 'n_units_0': 90, 'dropout_0': 0.22330471710255667, 'n_units_1': 75, 'dropout_1': 0.21877381597001344, 'n_units_2': 94, 'dropout_2': 0.26963192274000086, 'optimizer': 'SGD', 'lr': 1.918548588463614e-05}. Best is trial 1 with value: 0.0657.


Study statistics: 
  Number of finished trials:  9
  Number of complete trials:  9
Best trial:
  Value:  0.0657
  Params: 
    n_layers_conv: 3
    n_layers_dense: 1
    filters_0: 32
    kernel_size_0: 3
    strides_0: 1
    filters_1: 32
    kernel_size_1: 3
    strides_1: 1
    filters_2: 128
    kernel_size_2: 5
    strides_2: 1
    n_units_0: 82
    dropout_0: 0.2585625969071337
    optimizer: Adam
    lr: 0.00010247187950542526


# Q4_B
## Run 2 (without TimeOut)

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective)

    #pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    #print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-06-16 14:50:26,261] A new study created in memory with name: no-name-435cf050-910b-41dc-9908-a35998e53900


Files already downloaded and verified


[I 2022-06-16 14:51:28,442] Trial 0 finished with value: 0.01 and parameters: {'n_layers_conv': 3, 'n_layers_dense': 2, 'filters_0': 16, 'kernel_size_0': 5, 'strides_0': 2, 'filters_1': 16, 'kernel_size_1': 3, 'strides_1': 2, 'filters_2': 16, 'kernel_size_2': 5, 'strides_2': 2, 'n_units_0': 99, 'n_units_1': 81, 'optimizer': 'SGD', 'lr': 0.042299509527924294}. Best is trial 0 with value: 0.01.


Files already downloaded and verified


[I 2022-06-16 14:52:32,943] Trial 1 finished with value: 0.0529 and parameters: {'n_layers_conv': 2, 'n_layers_dense': 3, 'filters_0': 64, 'kernel_size_0': 5, 'strides_0': 1, 'filters_1': 16, 'kernel_size_1': 5, 'strides_1': 1, 'n_units_0': 72, 'n_units_1': 128, 'n_units_2': 72, 'optimizer': 'RMSprop', 'lr': 1.9636580084885864e-05}. Best is trial 1 with value: 0.0529.


...

Files already downloaded and verified


[I 2022-06-16 15:54:36,502] Trial 52 finished with value: 0.01 and parameters: {'n_layers_conv': 2, 'n_layers_dense': 1, 'filters_0': 64, 'kernel_size_0': 3, 'strides_0': 1, 'filters_1': 16, 'kernel_size_1': 5, 'strides_1': 1, 'n_units_0': 88, 'optimizer': 'RMSprop', 'lr': 0.09336995501676433}. Best is trial 11 with value: 0.09.


Files already downloaded and verified


[I 2022-06-16 15:55:38,272] Trial 53 finished with value: 0.0698 and parameters: {'n_layers_conv': 2, 'n_layers_dense': 1, 'filters_0': 64, 'kernel_size_0': 3, 'strides_0': 1, 'filters_1': 64, 'kernel_size_1': 5, 'strides_1': 1, 'n_units_0': 80, 'optimizer': 'RMSprop', 'lr': 8.016216899867066e-05}. Best is trial 11 with value: 0.09.


Files already downloaded and verified
